In [1]:
import os.path
import pandas as pd
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import DrawingOptions
from parameter_extractor import ParameterExtractor, Property

IPythonConsole.molSize = (300, 300)  # Change image size
DrawingOptions.bondLineWidth = 1.8

In [2]:
filename = "l_15"
path = f"./out/spe/{filename}_SPE.out"
path_no_rh = f"./out/spe_no_rh/{filename}_SPE_NoRh.out"
if not os.path.exists(path_no_rh):
    print(path_no_rh + " does not exist")

./out/spe_no_rh/l_15_SPE_NoRh.out does not exist


In [3]:
comp = ParameterExtractor(path=path, no_rh_path=path_no_rh)

E:\Data\PythonProjects\short_bridge\parameter_extractor.py:282: SyntaxWarning: invalid escape sequence '\s'
  pattern = f"\s+LP\s+\(\s+1\)\s*{atom_symbol}\s+{atom_idx}\s+s\("
E:\Data\PythonProjects\short_bridge\parameter_extractor.py:282: SyntaxWarning: invalid escape sequence '\s'
  pattern = f"\s+LP\s+\(\s+1\)\s*{atom_symbol}\s+{atom_idx}\s+s\("
E:\Data\PythonProjects\short_bridge\parameter_extractor.py:282: SyntaxWarning: invalid escape sequence '\s'
  pattern = f"\s+LP\s+\(\s+1\)\s*{atom_symbol}\s+{atom_idx}\s+s\("


FileNotFoundError: [Errno 2] No such file or directory: './out/spe/l_15_SPE.out'

In [8]:
bite_angle = comp.get_angle(comp.p1, comp.rh, comp.p2)
p1_rh_dist = comp.get_bond_distance(comp.p1, comp.rh)
p2_rh_dist = comp.get_bond_distance(comp.p2, comp.rh)
p1_bridge_dist = comp.get_bond_distance(comp.p1, comp.bridge)
p2_bridge_dist = comp.get_bond_distance(comp.p2, comp.bridge)
bridge_angle = comp.get_angle(comp.p1, comp.bridge, comp.p2)

print("P1-Rh: " + str(p1_rh_dist), "P2-Rh: " + str(p2_rh_dist), "P1-Bridge: " + str(p1_bridge_dist),
      "P2-Bridge: " + str(p2_bridge_dist), "Bite angle: " + str(bite_angle), "Bridge angle: " + str(bridge_angle))

P1-Rh: 2.3719312711164293 P2-Rh: 2.37194201200198 P1-Bridge: 1.8750391887104654 P2-Bridge: 1.8749946723657644 Bite angle: 72.70956883041772 Bridge angle: 97.16063672571833


In [9]:
comp.dipole

1.9588

In [10]:
r1_p1_r2_angle = comp.get_angle(comp.r1, comp.p1, comp.r2)
r1_p1_rh_angle = comp.get_angle(comp.r1, comp.p1, comp.rh)
rh_p1_bridge_angle = comp.get_angle(comp.rh, comp.p1, comp.bridge)
r1_p1_bridge_angle = comp.get_angle(comp.r1, comp.p1, comp.bridge)
r2_p1_bridge_angle = comp.get_angle(comp.r2, comp.p1, comp.bridge)
r2_p1_rh_angle = comp.get_angle(comp.r2, comp.p1, comp.rh)

p1_angle_sum = r1_p1_bridge_angle + r1_p1_r2_angle + r2_p1_bridge_angle
p1_angle_sum

317.8066693255203

In [11]:
r3_p2_r4_angle = comp.get_angle(comp.r3, comp.p2, comp.r4)
r3_p2_rh_angle = comp.get_angle(comp.r3, comp.p2, comp.rh)
rh_p2_bridge_angle = comp.get_angle(comp.rh, comp.p2, comp.bridge)
r3_p2_bridge_angle = comp.get_angle(comp.r3, comp.p2, comp.bridge)
r4_p2_bridge_angle = comp.get_angle(comp.r4, comp.p2, comp.bridge)
r4_p2_rh_angle = comp.get_angle(comp.r4, comp.p2, comp.rh)

p2_angle_sum = r3_p2_bridge_angle + r3_p2_r4_angle + r4_p2_bridge_angle
p2_angle_sum

317.80297812394565

In [4]:
comp.get_bond(comp.p1, comp.r1).get_prop(Property.ANTIBOND_OCCUPANCY)

'0.05997'

In [13]:
comp.p1.GetPropsAsDict()

{'nbo_charge': 1.1076,
 'nmr_isotropic': 319.369,
 'nmr_anisotropic': 118.7911,
 'molAtomMapNumber': 38,
 'nbo_charge_no_rh': 0.79838,
 'nmr_isotropic_no_rh': 241.6574,
 'nmr_anisotropic_no_rh': 93.1935,
 'lone_pair_occupancy_no_rh_1': 1.93141,
 'lone_pair_energy_no_rh_1': -0.33356}

In [13]:
if os.path.exists("parameters_phd.csv"):
    parameters_df = pd.read_csv("parameters_phd.csv")
else:
    parameters_df = pd.DataFrame()
param_list = []

for file in os.listdir("./out/spe/"):
    filename = file.split(".")[0]
    ligand_name = filename.replace("_SPE", "")
    path = f"./out/spe/{filename}.out"
    path_no_rh = f"./out/spe_no_rh/{filename}_NoRh.out"
    if not os.path.exists(path_no_rh):
        print(path_no_rh + " does not exist")
        continue

    if ligand_name in parameters_df["ligand"].values:
        print(f"{ligand_name} already parsed, skipping...")
        continue
        
    print(ligand_name + ": Parameter extraction started.")
    comp = ParameterExtractor(path=path, no_rh_path=path_no_rh)
    
    r1_p1_r2_angle = comp.get_angle(comp.r1, comp.p1, comp.r2)
    r1_p1_rh_angle = comp.get_angle(comp.r1, comp.p1, comp.rh)
    rh_p1_bridge_angle = comp.get_angle(comp.rh, comp.p1, comp.bridge)
    r1_p1_bridge_angle = comp.get_angle(comp.r1, comp.p1, comp.bridge)
    r2_p1_bridge_angle = comp.get_angle(comp.r2, comp.p1, comp.bridge)
    r2_p1_rh_angle = comp.get_angle(comp.r2, comp.p1, comp.rh)
    
    p1_angle_sum = r1_p1_bridge_angle + r1_p1_r2_angle + r2_p1_bridge_angle
    
    r3_p2_r4_angle = comp.get_angle(comp.r3, comp.p2, comp.r4)
    r3_p2_rh_angle = comp.get_angle(comp.r3, comp.p2, comp.rh)
    rh_p2_bridge_angle = comp.get_angle(comp.rh, comp.p2, comp.bridge)
    r3_p2_bridge_angle = comp.get_angle(comp.r3, comp.p2, comp.bridge)
    r4_p2_bridge_angle = comp.get_angle(comp.r4, comp.p2, comp.bridge)
    r4_p2_rh_angle = comp.get_angle(comp.r4, comp.p2, comp.rh)
    
    p2_angle_sum = r3_p2_bridge_angle + r3_p2_r4_angle + r4_p2_bridge_angle
    param_dict = {
        # Global parameters
        "ligand": ligand_name,
        "homo": comp.homo,
        "lumo": comp.lumo,
        "dipole": comp.dipole,
        "rh_charge": comp.rh.get_prop(Property.CHARGE),
        "rh_LP_1_occ": comp.rh.get_prop(Property.LONE_PAIR_OCCUPANCY_1),
        "rh_LP_1_eng": comp.rh.get_prop(Property.LONE_PAIR_ENERGY_1),
        "rh_LP_2_occ": comp.rh.get_prop(Property.LONE_PAIR_OCCUPANCY_2),
        "rh_LP_2_eng": comp.rh.get_prop(Property.LONE_PAIR_ENERGY_2),
        "rh_LP_3_occ": comp.rh.get_prop(Property.LONE_PAIR_OCCUPANCY_3),
        "rh_LP_3_eng": comp.rh.get_prop(Property.LONE_PAIR_ENERGY_3),
        "rh_LP_4_occ": comp.rh.get_prop(Property.LONE_PAIR_OCCUPANCY_4),
        "rh_LP_4_eng": comp.rh.get_prop(Property.LONE_PAIR_ENERGY_4),
        "bite_angle": comp.get_angle(comp.p1, comp.bridge, comp.p2),
        "homo_no_rh": comp.homo_no_rh,
        "lumo_no_rh": comp.lumo_no_rh,
        "dipole_no_rh": comp.dipole_no_rh,
        # missing polarizability, I don't know where to get it from, but does not seem important
        
        # C1 parameters
        "r1_nbo": comp.r1.get_prop(Property.CHARGE),
        "r2_nbo": comp.r2.get_prop(Property.CHARGE),
        "r3_nbo": comp.r3.get_prop(Property.CHARGE),
        "r4_nbo": comp.r4.get_prop(Property.CHARGE),
        "bridge_nbo": comp.bridge.get_prop(Property.CHARGE),
        "p1_nbo": comp.p1.get_prop(Property.CHARGE),
        "p2_nbo": comp.p2.get_prop(Property.CHARGE),
        "c1_nbo": comp.c1.get_prop(Property.CHARGE),
        "c2_nbo": comp.c2.get_prop(Property.CHARGE),
        "c3_nbo": comp.c3.get_prop(Property.CHARGE),
        "c4_nbo": comp.c4.get_prop(Property.CHARGE),
        
        "p1_nmr": comp.p1.get_prop(Property.NMR_ISOTROPIC),
        "p1_nmr_aniso": comp.p1.get_prop(Property.NMR_ANISOTROPIC),
        "p2_nmr": comp.p2.get_prop(Property.NMR_ISOTROPIC),
        "p2_nmr_aniso": comp.p2.get_prop(Property.NMR_ANISOTROPIC),
        
        "p1_rh_bond_occ": comp.get_bond(comp.p1, comp.rh).get_prop(Property.BOND_OCCUPANCY),
        "p2_rh_bond_occ": comp.get_bond(comp.p2, comp.rh).get_prop(Property.BOND_OCCUPANCY),
        "p1_rh_bond_eng": comp.get_bond(comp.p1, comp.rh).get_prop(Property.BOND_ENERGY),
        "p2_rh_bond_eng": comp.get_bond(comp.p2, comp.rh).get_prop(Property.BOND_ENERGY),
        "p1_rh_antibond_occ": comp.get_bond(comp.p1, comp.rh).get_prop(Property.ANTIBOND_OCCUPANCY),
        "p2_rh_antibond_occ": comp.get_bond(comp.p2, comp.rh).get_prop(Property.ANTIBOND_OCCUPANCY),
        "p1_rh_antibond_eng": comp.get_bond(comp.p1, comp.rh).get_prop(Property.ANTIBOND_ENERGY),
        "p2_rh_antibond_eng": comp.get_bond(comp.p2, comp.rh).get_prop(Property.ANTIBOND_ENERGY),
        "p1_r1_bond_occ": comp.get_bond(comp.p1, comp.r1).get_prop(Property.BOND_OCCUPANCY),
        "p1_r2_bond_occ": comp.get_bond(comp.p1, comp.r2).get_prop(Property.BOND_OCCUPANCY),
        "p2_r3_bond_occ": comp.get_bond(comp.p2, comp.r3).get_prop(Property.BOND_OCCUPANCY),
        "p2_r4_bond_occ": comp.get_bond(comp.p2, comp.r4).get_prop(Property.BOND_OCCUPANCY),
        "p1_r1_bond_eng": comp.get_bond(comp.p1, comp.r1).get_prop(Property.BOND_ENERGY),
        "p1_r2_bond_eng": comp.get_bond(comp.p1, comp.r2).get_prop(Property.BOND_ENERGY),
        "p2_r3_bond_eng": comp.get_bond(comp.p2, comp.r3).get_prop(Property.BOND_ENERGY),
        "p2_r4_bond_eng": comp.get_bond(comp.p2, comp.r4).get_prop(Property.BOND_ENERGY),
        "p1_r1_antibond_occ": comp.get_bond(comp.p1, comp.r1).get_prop(Property.ANTIBOND_OCCUPANCY),
        "p1_r2_antibond_occ": comp.get_bond(comp.p1, comp.r2).get_prop(Property.ANTIBOND_OCCUPANCY),
        "p2_r3_antibond_occ": comp.get_bond(comp.p2, comp.r3).get_prop(Property.ANTIBOND_OCCUPANCY),
        "p2_r4_antibond_occ": comp.get_bond(comp.p2, comp.r4).get_prop(Property.ANTIBOND_OCCUPANCY),
        "p1_r1_antibond_eng": comp.get_bond(comp.p1, comp.r1).get_prop(Property.ANTIBOND_ENERGY),
        "p1_r2_antibond_eng": comp.get_bond(comp.p1, comp.r2).get_prop(Property.ANTIBOND_ENERGY),
        "p2_r3_antibond_eng": comp.get_bond(comp.p2, comp.r3).get_prop(Property.ANTIBOND_ENERGY),
        "p2_r4_antibond_eng": comp.get_bond(comp.p2, comp.r4).get_prop(Property.ANTIBOND_ENERGY),
        "p1_bridge_bond_occ": comp.get_bond(comp.p1, comp.bridge).get_prop(Property.BOND_OCCUPANCY),
        "p2_bridge_bond_occ": comp.get_bond(comp.p2, comp.bridge).get_prop(Property.BOND_OCCUPANCY),
        "p1_bridge_bond_eng": comp.get_bond(comp.p1, comp.bridge).get_prop(Property.BOND_ENERGY),
        "p2_bridge_bond_eng": comp.get_bond(comp.p2, comp.bridge).get_prop(Property.BOND_ENERGY),
        "p1_bridge_antibond_occ": comp.get_bond(comp.p1, comp.bridge).get_prop(Property.ANTIBOND_OCCUPANCY),
        "p2_bridge_antibond_occ": comp.get_bond(comp.p2, comp.bridge).get_prop(Property.ANTIBOND_OCCUPANCY),
        "p1_bridge_antibond_eng": comp.get_bond(comp.p1, comp.bridge).get_prop(Property.ANTIBOND_ENERGY),
        "p2_bridge_antibond_eng": comp.get_bond(comp.p2, comp.bridge).get_prop(Property.ANTIBOND_ENERGY),
        
        "c1_rh_dist": comp.get_bond_distance(comp.c1, comp.rh),
        "c2_rh_dist": comp.get_bond_distance(comp.c2, comp.rh),
        "c3_rh_dist": comp.get_bond_distance(comp.c3, comp.rh),
        "c4_rh_dist": comp.get_bond_distance(comp.c4, comp.rh),
        "p1_c1_dist": comp.get_bond_distance(comp.p1, comp.c1),
        "p1_c2_dist": comp.get_bond_distance(comp.p1, comp.c2),
        "p2_c3_dist": comp.get_bond_distance(comp.p2, comp.c3),
        "p2_c4_dist": comp.get_bond_distance(comp.p2, comp.c4),
        "p1_bridge_dist": comp.get_bond_distance(comp.p1, comp.bridge),
        "p2_bridge_dist": comp.get_bond_distance(comp.p2, comp.bridge),
        
        "p1_angle_sum": p1_angle_sum,
        "p2_angle_sum": p2_angle_sum,
        "rh_p1_bridge_angle": rh_p1_bridge_angle,
        "rh_p2_bridge_angle": rh_p2_bridge_angle,
        "r1_p1_rh_angle": r1_p1_rh_angle,
        "r2_p1_rh_angle": r2_p1_rh_angle,
        "r3_p2_rh_angle": r3_p2_rh_angle,
        "r4_p2_rh_angle": r4_p2_rh_angle,
        "r1_p1_bridge_angle": r1_p1_bridge_angle,
        "r2_p1_bridge_angle": r2_p1_bridge_angle,
        "r3_p2_bridge_angle": r3_p2_bridge_angle,
        "r4_p2_bridge_angle": r4_p2_bridge_angle,
        
        "p1_nbo_no_rh": comp.p1.get_prop(Property.CHARGE_NO_RH),
        "p2_nbo_no_rh": comp.p2.get_prop(Property.CHARGE_NO_RH),
        "r1_nbo_no_rh": comp.r1.get_prop(Property.CHARGE_NO_RH),
        "r2_nbo_no_rh": comp.r2.get_prop(Property.CHARGE_NO_RH),
        "r3_nbo_no_rh": comp.r3.get_prop(Property.CHARGE_NO_RH),
        "r4_nbo_no_rh": comp.r4.get_prop(Property.CHARGE_NO_RH),
        "bridge_nbo_no_rh": comp.bridge.get_prop(Property.CHARGE_NO_RH),
        
        "p1_nmr_no_rh": comp.p1.get_prop(Property.NMR_ISOTROPIC_NO_RH),
        "p1_nmr_aniso_no_rh": comp.p1.get_prop(Property.NMR_ANISOTROPIC_NO_RH),
        "p2_nmr_no_rh": comp.p2.get_prop(Property.NMR_ISOTROPIC_NO_RH),
        "p2_nmr_aniso_no_rh": comp.p2.get_prop(Property.NMR_ANISOTROPIC_NO_RH),
        
        "p1_LP_occ_no_rh": comp.p1.get_prop(Property.LONE_PAIR_OCCUPANCY_NO_RH_1),
        "p1_LP_eng_no_rh": comp.p1.get_prop(Property.LONE_PAIR_ENERGY_NO_RH_1),
        "p2_LP_occ_no_rh": comp.p2.get_prop(Property.LONE_PAIR_OCCUPANCY_NO_RH_1),
        "p2_LP_eng_no_rh": comp.p2.get_prop(Property.LONE_PAIR_ENERGY_NO_RH_1),
        
        "p1_r1_bond_occ_no_rh": comp.get_bond(comp.p1, comp.r1).get_prop(Property.BOND_OCCUPANCY_NO_RH),
        "p1_r2_bond_occ_no_rh": comp.get_bond(comp.p1, comp.r2).get_prop(Property.BOND_OCCUPANCY_NO_RH),
        "p2_r3_bond_occ_no_rh": comp.get_bond(comp.p2, comp.r3).get_prop(Property.BOND_OCCUPANCY_NO_RH),
        "p2_r4_bond_occ_no_rh": comp.get_bond(comp.p2, comp.r4).get_prop(Property.BOND_OCCUPANCY_NO_RH),
        "p1_r1_bond_eng_no_rh": comp.get_bond(comp.p1, comp.r1).get_prop(Property.BOND_ENERGY_NO_RH),
        "p1_r2_bond_eng_no_rh": comp.get_bond(comp.p1, comp.r2).get_prop(Property.BOND_ENERGY_NO_RH),
        "p2_r3_bond_eng_no_rh": comp.get_bond(comp.p2, comp.r3).get_prop(Property.BOND_ENERGY_NO_RH),
        "p2_r4_bond_eng_no_rh": comp.get_bond(comp.p2, comp.r4).get_prop(Property.BOND_ENERGY_NO_RH),
        "p1_r1_antibond_occ_no_rh": comp.get_bond(comp.p1, comp.r1).get_prop(Property.ANTIBOND_OCCUPANCY_NO_RH),
        "p1_r2_antibond_occ_no_rh": comp.get_bond(comp.p1, comp.r2).get_prop(Property.ANTIBOND_OCCUPANCY_NO_RH),
        "p2_r3_antibond_occ_no_rh": comp.get_bond(comp.p2, comp.r3).get_prop(Property.ANTIBOND_OCCUPANCY_NO_RH),
        "p2_r4_antibond_occ_no_rh": comp.get_bond(comp.p2, comp.r4).get_prop(Property.ANTIBOND_OCCUPANCY_NO_RH),
        "p1_r1_antibond_eng_no_rh": comp.get_bond(comp.p1, comp.r1).get_prop(Property.ANTIBOND_ENERGY_NO_RH),
        "p1_r2_antibond_eng_no_rh": comp.get_bond(comp.p1, comp.r2).get_prop(Property.ANTIBOND_ENERGY_NO_RH),
        "p2_r3_antibond_eng_no_rh": comp.get_bond(comp.p2, comp.r3).get_prop(Property.ANTIBOND_ENERGY_NO_RH),
        "p2_r4_antibond_eng_no_rh": comp.get_bond(comp.p2, comp.r4).get_prop(Property.ANTIBOND_ENERGY_NO_RH),
        "p1_bridge_bond_occ_no_rh": comp.get_bond(comp.p1, comp.bridge).get_prop(Property.BOND_OCCUPANCY_NO_RH),
        "p2_bridge_bond_occ_no_rh": comp.get_bond(comp.p2, comp.bridge).get_prop(Property.BOND_OCCUPANCY_NO_RH),
        "p1_bridge_bond_eng_no_rh": comp.get_bond(comp.p1, comp.bridge).get_prop(Property.BOND_ENERGY_NO_RH),
        "p2_bridge_bond_eng_no_rh": comp.get_bond(comp.p2, comp.bridge).get_prop(Property.BOND_ENERGY_NO_RH),
        "p1_bridge_antibond_occ_no_rh": comp.get_bond(comp.p1, comp.bridge).get_prop(Property.ANTIBOND_OCCUPANCY_NO_RH),
        "p2_bridge_antibond_occ_no_rh": comp.get_bond(comp.p2, comp.bridge).get_prop(Property.ANTIBOND_OCCUPANCY_NO_RH),
        "p1_bridge_antibond_eng_no_rh": comp.get_bond(comp.p1, comp.bridge).get_prop(Property.ANTIBOND_ENERGY_NO_RH),
        "p2_bridge_antibond_eng_no_rh": comp.get_bond(comp.p2, comp.bridge).get_prop(Property.ANTIBOND_ENERGY_NO_RH),
        
        "p1_LP_s_occ_no_rh": comp.p1.get_prop(Property.LONE_PAIR_S_ORBITAL_NO_RH),
        "p2_LP_s_occ_no_rh": comp.p2.get_prop(Property.LONE_PAIR_S_ORBITAL_NO_RH),
    }
    
    for radius in range(1, 7, 1):
        bv = comp.buried_volume(radius)
        bv.octant_analysis()
        param_dict[f"Vbur%_{float(radius)}"] = bv.fraction_buried_volume * 100
        param_dict[f"NE_{float(radius)}"] = bv.quadrants["percent_buried_volume"][1]
        param_dict[f"NW_{float(radius)}"] = bv.quadrants["percent_buried_volume"][2]
        param_dict[f"SE_{float(radius)}"] = bv.quadrants["percent_buried_volume"][3]
        param_dict[f"SW_{float(radius)}"] = bv.quadrants["percent_buried_volume"][4]
        
    param_list.append(param_dict)

parameters_df = pd.concat([parameters_df, pd.DataFrame(param_list)])
print(f"{len(param_list)} new ligands added to the DataFrame")

adPAMP: Parameter extraction started.
cyPAMP already parsed, skipping...
dcPAMP already parsed, skipping...
dcpa already parsed, skipping...
diprpa already parsed, skipping...
DS-10 already parsed, skipping...
DS-11 already parsed, skipping...
DS-12 already parsed, skipping...
DS-13 already parsed, skipping...
DS-14 already parsed, skipping...
DS-15 already parsed, skipping...
DS-16 already parsed, skipping...
DS-17 already parsed, skipping...
DS-18 already parsed, skipping...
DS-19 already parsed, skipping...
DS-1 already parsed, skipping...
DS-20 already parsed, skipping...
DS-21 already parsed, skipping...
DS-22 already parsed, skipping...
DS-23 already parsed, skipping...
DS-24 already parsed, skipping...
DS-25 already parsed, skipping...
DS-26 already parsed, skipping...
DS-27 already parsed, skipping...
DS-28 already parsed, skipping...
DS-29 already parsed, skipping...
DS-2 already parsed, skipping...
DS-30 already parsed, skipping...
DS-31 already parsed, skipping...
DS-32 alre

In [15]:
parameters_df

,Unnamed: 0,ligand,homo,lumo,dipole,rh_charge,rh_LP_1_occ,rh_LP_1_eng,rh_LP_2_occ,rh_LP_2_eng,...,Vbur%_5.0,NE_5.0,NW_5.0,SE_5.0,SW_5.0,Vbur%_6.0,NE_6.0,NW_6.0,SE_6.0,SW_6.0
0,0.0,cyPAMP,-0.30367,-0.15834,1.6275,-0.37258,1.98066,-0.30583,1.95467,-0.35714,...,42.874909,44.303239,46.613951,38.696351,41.886096,38.688149,42.962039,41.986031,35.033335,34.771191
1,1.0,dcPAMP,-0.30529,-0.16003,0.4737,-0.35437,1.98307,-0.30775,1.95815,-0.35837,...,47.315009,47.872189,47.901359,43.261802,50.224685,40.866479,40.639485,40.905710,39.863032,42.057690
2,2.0,dcpa,-0.30559,-0.15670,0.4496,-0.37791,1.98391,-0.30623,1.94745,-0.35936,...,46.662633,45.645833,47.370786,46.110971,47.522942,41.961087,40.722935,42.928024,41.100277,43.093111
3,3.0,diprpa,-0.31343,-0.16420,1.7617,-0.38593,1.98416,-0.31577,1.94642,-0.36749,...,45.769804,44.875595,46.616316,44.825140,46.762165,37.797066,37.065173,38.606286,36.490544,39.026260
4,4.0,DS-10,-0.30313,-0.17047,1.5365,-0.39825,1.98238,-0.30666,1.95305,-0.35787,...,41.628299,40.666488,42.586169,40.668853,42.591687,37.426641,37.590367,37.261554,37.592635,37.262007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,289.0,MS-92,-0.29683,-0.15800,2.1256,-0.34873,1.98334,-0.30299,1.95312,-0.35436,...,48.356446,48.915203,48.222226,44.351329,51.937025,42.384462,43.225090,41.522518,39.802259,44.987981
290,290.0,MS-93,-0.30480,-0.15955,1.8883,-0.3058,1.9823,-0.30363,1.95876,-0.35794,...,51.315789,48.516288,48.936489,57.096118,50.714263,46.428069,41.051295,41.059005,59.026260,44.575718
291,291.0,MS-9,-0.31194,-0.16819,1.1081,-0.40772,1.98036,-0.31234,1.9542,-0.36659,...,40.612879,37.801552,43.537731,42.235344,38.876888,34.157218,29.438523,37.134564,37.092839,32.962946
292,292.0,phPAMP,-0.30520,-0.16204,0.9453,-0.39322,1.98101,-0.30719,1.95371,-0.3605,...,42.189422,43.903535,44.068304,39.535965,41.249882,37.164270,42.506690,38.247540,33.453671,34.449181


In [16]:
parameters_df.to_csv("parameters_phd.csv", index=False)

In [25]:
file_path = "parameters_phd.csv"
df = pd.read_csv(file_path)

In [28]:
list(df.columns)

['ligand',
 'homo',
 'lumo',
 'dipole',
 'rh_charge',
 'rh_LP_1_occ',
 'rh_LP_1_eng',
 'rh_LP_2_occ',
 'rh_LP_2_eng',
 'rh_LP_3_occ',
 'rh_LP_3_eng',
 'rh_LP_4_occ',
 'rh_LP_4_eng',
 'bite_angle',
 'homo_no_rh',
 'lumo_no_rh',
 'dipole_no_rh',
 'r1_nbo',
 'r2_nbo',
 'r3_nbo',
 'r4_nbo',
 'bridge_nbo',
 'p1_nbo',
 'p2_nbo',
 'c1_nbo',
 'c2_nbo',
 'c3_nbo',
 'c4_nbo',
 'p1_nmr',
 'p1_nmr_aniso',
 'p2_nmr',
 'p2_nmr_aniso',
 'p1_rh_bond_occ',
 'p2_rh_bond_occ',
 'p1_rh_bond_eng',
 'p2_rh_bond_eng',
 'p1_rh_antibond_occ',
 'p2_rh_antibond_occ',
 'p1_rh_antibond_eng',
 'p2_rh_antibond_eng',
 'p1_r1_bond_occ',
 'p1_r2_bond_occ',
 'p2_r3_bond_occ',
 'p2_r4_bond_occ',
 'p1_r1_bond_eng',
 'p1_r2_bond_eng',
 'p2_r3_bond_eng',
 'p2_r4_bond_eng',
 'p1_r1_antibond_occ',
 'p1_r2_antibond_occ',
 'p2_r3_antibond_occ',
 'p2_r4_antibond_occ',
 'p1_r1_antibond_eng',
 'p1_r2_antibond_eng',
 'p2_r3_antibond_eng',
 'p2_r4_antibond_eng',
 'p1_bridge_bond_occ',
 'p2_bridge_bond_occ',
 'p1_bridge_bond_eng',
 '